In [16]:
import numpy as np
import pandas as pd
from re import sub
from time import time
import pickle

from comet_ml import Experiment

from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras_tqdm import TQDMNotebookCallback as ktqdm
from keras.utils import normalize
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import TensorBoard
from keras.optimizers import Adam, SGD
from keras.regularizers import l1, l2
from keras.initializers import RandomUniform, RandomNormal
from keras.layers.advanced_activations import LeakyReLU

from tensorflow.nn import relu, softmax

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler, Normalizer
from sklearn.impute import SimpleImputer
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error, r2_score

import scipy.stats as st

import seaborn as sns

import matplotlib.pyplot as plt

In [2]:
experiment = Experiment(api_key="CndJ3YmXyZcxmsV8EccJEuu9C",
                        project_name="Regression_training_features", workspace="paologalligit")

COMET INFO: old comet version (1.0.56) detected. current: 2.0.2 please update your comet lib with command: `pip install --no-cache-dir --upgrade comet_ml`
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/paologalligit/regression-training-features/83a4085ccce74317b3c3b0fa1bb37d3e



In [2]:
def print_dict(d):
    print("{:<15} {:<15} {:<10}".format('NeuralNetwork','MEAN','STD'))
    for k, v in d.items():
        print("{:<15} {:<15} {:<10}".format(k, round(v['MEAN'], 4), round(v['STD'], 4)))

In [18]:
df = pd.read_csv('fifa19.csv')

In [19]:
df.drop(columns=['Unnamed: 0', 'ID', 'Photo', 'Flag', 'Club Logo', 'Real Face', 'Preferred Foot',
                 'Body Type', 'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until', 
                 'Name', 'Club', 'Nationality', 'Work Rate', 'Position'],inplace=True)

## Obiettivo: predire valore dei giocatori

Pre-processing: convertire value, wage e release clause da string a float

In [20]:
curs=["Release Clause", "Value", "Wage"]

for cur in curs:
    def curr2val(x):
        x = str(x).replace('€', '')
        if 'K' in x: x = float(str(x).replace('K', '')) * 1000
        else: x = float(str(x).replace('M', '')) * 1000000
        return x
    df[cur] = df[cur].apply(curr2val)
    

Individuare eventuali outlier nella colonna value

In [21]:
def detect_outlier(data, threshold = 3):
    outliers=[]
    mean = np.mean(data)
    std = np.std(data)
    
    for y in data:
        score= (y - mean) / std 
        if np.abs(score) > threshold:
            outliers.append(y)
    return outliers

In [22]:
min_out = min(detect_outlier(df['Value'], threshold = 2))

df = df[df['Value'] < min_out] 
df = df[df['Value'] > 0]

Conversione in interi per le altre label

In [23]:
cols=["LS", "ST", "RS", "LW", "LF", "CF", "RF", "RW","LAM", "CAM", "RAM", "LM", "LCM", "CM", "RCM", "RM", "LWB", "LDM","CDM", "RDM", "RWB", "LB", "LCB", "CB", "RCB", "RB"]
for col in cols:
    df[col] = df[col].str[:-2]
    df[col] = df[col].astype(float)

In [24]:
df['Height'] = df['Height'].str.replace("'",'.')
df['Height'] = df['Height'].astype(float)

df['Weight'] = df['Weight'].str[:-3]
df['Weight'] = df['Weight'].astype(float)

Mescolo le righe del dataset

In [25]:
df_flt = df.sample(frac=1)

train_slice = int(len(df_flt) * 0.8)

train = df_flt[:train_slice]
test = df_flt[train_slice:]

In [26]:
df_corr = train.corr()

labels = []
for label in df_corr:
#     if df_corr['Value'][label] < 0 or df_corr['Value'][label] > 0.5: labels.append(label)
    if df_corr['Value'][label] > 0.55: labels.append(label)
        
train = train[labels] 
test = test[labels]
train.head() 

,Overall,Potential,Value,Wage,LCM,CM,RCM,Reactions,Release Clause
14788,60,71,350000.0,2000.0,55.0,55.0,55.0,60.0,NaN
17902,51,63,60000.0,1000.0,46.0,46.0,46.0,36.0,117000.0
6090,69,81,1800000.0,5000.0,62.0,62.0,62.0,56.0,NaN
9371,66,66,675000.0,1000.0,50.0,50.0,50.0,60.0,911000.0
7144,68,68,600000.0,3000.0,51.0,51.0,51.0,59.0,900000.0


In [27]:
y_train = train.loc[:, ['Value']]
X_train = train.drop(columns='Value')

y_test = test.loc[:, ['Value']]
X_test = test.drop(columns='Value')

Sostiuisco eventuali valori nan con la media della colonna

In [28]:
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(X_train)
X_full = imputer.transform(X_train)

imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(y_train)
y_full = imputer.transform(y_train)

imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(X_test)
X_test_full = imputer.transform(X_test)

imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(y_test)
y_test_full = imputer.transform(y_test)

Scalo i valori, sia per i caratteri che per il target

In [29]:
# scaler = RobustScaler()
# scaler = scaler.fit(X_train)
# X_train = scaler.transform(X_train)

# X_train_scaled = preprocessing.scale(X_train)
scaler = StandardScaler().fit(X_full)
X_scaled = scaler.transform(X_full)

scaler_train = StandardScaler().fit(y_full)
y_scaled = scaler_train.transform(y_full) 
# X_train_scaled, X_test_scaled

scaler = StandardScaler().fit(X_test_full)
X_test_scaled = scaler.transform(X_test_full)

scaler_test = StandardScaler().fit(y_test_full)
y_test_scaled = scaler_test.transform(y_test_full) 

In [30]:
data = {
    'X_scaled': X_scaled,
    'y_scaled': y_scaled,
    'X_test_scaled': X_test_scaled,
    'y_test_scaled': y_test_scaled,
    'y_test_full': y_test_full,
}

with open('dataset_corr_labels', 'wb') as f:
    pickle.dump(data, f)

In [16]:
def coeff_determination(y_test, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square( y_test-y_pred ))
    SS_tot = K.sum(K.square( y_test - K.mean(y_test) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()))

In [17]:
ts_board = TensorBoard(log_dir='value_predictions_v3/{}'.format('kfold_' + str(time())))

In [18]:
def mean_abs_error(prediction, target):
    res = 0
    tot = 0
    for i in range(len(target)):
        if target[i][0] != 0 and not np.isnan(prediction[i][0]):
#             print(prediction[i][0], target[i][0])
            res += abs(prediction[i][0] - target[i][0])
            tot += 1
#     print(tot, len(target), len(prediction))
    return round(res / tot, 2)

# Cross-validation
### K-Fold

In [19]:
# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
outer_kfold = KFold(n_splits=3, random_state=42, shuffle=False)
inner_kfold = KFold(n_splits=5, random_state=19, shuffle=False )

In [20]:
def build_nn(input_layer, neurons):
    model = Sequential()

    model.add(Dense(neurons, input_dim = input_layer, activation='relu'))

    model.add(Dense(1, activation = 'linear'))
    
    return model

In [21]:
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

Instructions for updating:
Colocations handled automatically by placer.


In [22]:
total_results = []

for fold, (train_index_outer, test_index_outer) in enumerate(outer_kfold.split(X_scaled)):
    
    X_train_outer, X_test_outer = X_scaled[train_index_outer], X_scaled[test_index_outer]
    y_train_outer, y_test_outer = y_scaled[train_index_outer], y_scaled[test_index_outer]
    
    inner_mean_scores = []

    neurons = [5, 10, 20, 35, 50, 75, 100]
    
    for neuron in neurons:
        
        inner_scores = []
        
        for train_index_inner, test_index_inner in inner_kfold.split(X_train_outer):
            X_train_inner, X_test_inner = X_train_outer[train_index_inner], X_train_outer[test_index_inner]
            y_train_inner, y_test_inner = y_train_outer[train_index_inner], y_train_outer[test_index_inner]

            nn = build_nn(X_train_inner.shape[1], neuron)
            nn.compile(optimizer=opt, loss='mean_absolute_error', metrics=[coeff_determination])
            nn.fit(X_train_inner, y_train_inner, batch_size=125, epochs=100, shuffle=True, verbose=1)

            res = nn.predict(X_test_inner)
            loss, acc = nn.evaluate(X_train_inner, y_train_inner, verbose=1)
            
            reversed_res = scaler_train.inverse_transform(res)
            reversed_label = scaler_train.inverse_transform(y_test_inner)
            inner_scores.append(
                {'ACC': acc * 100, 'MAE': mean_abs_error(reversed_res, reversed_label), 'MAX': max(reversed_label), 'MEAN': round(np.mean(reversed_label), 4)}
            )
            
            print("loss: %.4f, acc: %.4f%%" % (loss, acc*100))
        
        inner_mean_scores.append({'n_{}'.format(neuron): inner_scores})
    total_results.append({'fold_{}'.format(fold): inner_mean_scores})

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
7428/7428 [==============================] - 0s 19us/step - loss: 0.4816 - coeff_determination: 0.5777
Epoch 2/100
7428/7428 [==============================] - 0s 11us/step - loss: 0.3684 - coeff_determination: 0.7238
Epoch 3/100
7428/7428 [==============================] - 0s 11us/step - loss: 0.2752 - coeff_determination: 0.8237
Epoch 4/100
7428/7428 [==============================] - 0s 10us/step - loss: 0.2047 - coeff_determination: 0.8772
Epoch 5/100
7428/7428 [==============================] - 0s 11us/step - loss: 0.1666 - coeff_determination: 0.9063
Epoch 6/100
7428/7428 [==============================] - 0s 12us/step - loss: 0.1454 - coeff_determination: 0.9192
Epoch 7/100
7428/7428 [==============================] - 0s 11us/step - loss: 0.1326 - coeff_determination: 0.9274
Epoch 8/100
7428/7428 [==============================] - 0s 11us/step - loss: 0.1245 - coeff_determination: 0.9321
Epoch 9/100
7428/7428 [=========

7428/7428 [==============================] - 0s 9us/step - loss: 0.1002 - coeff_determination: 0.9324
Epoch 40/100
7428/7428 [==============================] - 0s 10us/step - loss: 0.1000 - coeff_determination: 0.9321
Epoch 41/100
7428/7428 [==============================] - 0s 8us/step - loss: 0.0996 - coeff_determination: 0.9302
Epoch 42/100
7428/7428 [==============================] - 0s 8us/step - loss: 0.0992 - coeff_determination: 0.9318
Epoch 43/100
7428/7428 [==============================] - 0s 9us/step - loss: 0.0993 - coeff_determination: 0.9316
Epoch 44/100
7428/7428 [==============================] - 0s 13us/step - loss: 0.0988 - coeff_determination: 0.9281
Epoch 45/100
7428/7428 [==============================] - 0s 12us/step - loss: 0.0988 - coeff_determination: 0.9275
Epoch 46/100
7428/7428 [==============================] - 0s 9us/step - loss: 0.0985 - coeff_determination: 0.9305
Epoch 47/100
7428/7428 [==============================] - 0s 8us/step - loss: 0.0984 - coe

7428/7428 [==============================] - 0s 9us/step - loss: 0.0977 - coeff_determination: 0.9300
Epoch 81/100
7428/7428 [==============================] - 0s 8us/step - loss: 0.0979 - coeff_determination: 0.9307
Epoch 82/100
7428/7428 [==============================] - 0s 8us/step - loss: 0.0976 - coeff_determination: 0.9332
Epoch 83/100
7428/7428 [==============================] - 0s 8us/step - loss: 0.0975 - coeff_determination: 0.9327
Epoch 84/100
7428/7428 [==============================] - 0s 11us/step - loss: 0.0973 - coeff_determination: 0.9355
Epoch 85/100
7428/7428 [==============================] - 0s 9us/step - loss: 0.0976 - coeff_determination: 0.9320
Epoch 86/100
7428/7428 [==============================] - 0s 8us/step - loss: 0.0972 - coeff_determination: 0.9305
Epoch 87/100
7428/7428 [==============================] - 0s 8us/step - loss: 0.0972 - coeff_determination: 0.9340
Epoch 88/100
7428/7428 [==============================] - 0s 9us/step - loss: 0.0971 - coeff

7428/7428 [==============================] - 0s 13us/step - loss: 0.1018 - coeff_determination: 0.9197
Epoch 21/100
7428/7428 [==============================] - 0s 8us/step - loss: 0.1018 - coeff_determination: 0.9172
Epoch 22/100
7428/7428 [==============================] - 0s 8us/step - loss: 0.1013 - coeff_determination: 0.9181
Epoch 23/100
7428/7428 [==============================] - 0s 9us/step - loss: 0.1010 - coeff_determination: 0.9194
Epoch 24/100
7428/7428 [==============================] - 0s 10us/step - loss: 0.1012 - coeff_determination: 0.9187
Epoch 25/100
7428/7428 [==============================] - 0s 10us/step - loss: 0.1006 - coeff_determination: 0.9210
Epoch 26/100
7428/7428 [==============================] - 0s 9us/step - loss: 0.1004 - coeff_determination: 0.9165
Epoch 27/100
7428/7428 [==============================] - 0s 9us/step - loss: 0.1003 - coeff_determination: 0.9232
Epoch 28/100
7428/7428 [==============================] - 0s 9us/step - loss: 0.1001 - coe

7428/7428 [==============================] - 0s 9us/step - loss: 0.0923 - coeff_determination: 0.9287
Epoch 61/100
7428/7428 [==============================] - 0s 9us/step - loss: 0.0923 - coeff_determination: 0.9313
Epoch 62/100
7428/7428 [==============================] - 0s 9us/step - loss: 0.0922 - coeff_determination: 0.9289
Epoch 63/100
7428/7428 [==============================] - 0s 9us/step - loss: 0.0925 - coeff_determination: 0.9277
Epoch 64/100
7428/7428 [==============================] - 0s 9us/step - loss: 0.0922 - coeff_determination: 0.9271
Epoch 65/100
7428/7428 [==============================] - 0s 9us/step - loss: 0.0923 - coeff_determination: 0.9300
Epoch 66/100
7428/7428 [==============================] - 0s 10us/step - loss: 0.0921 - coeff_determination: 0.9271
Epoch 67/100
7428/7428 [==============================] - 0s 9us/step - loss: 0.0920 - coeff_determination: 0.9294
Epoch 68/100
7428/7428 [==============================] - 0s 9us/step - loss: 0.0920 - coeff

...

7428/7428 [==============================] - 0s 31us/step - loss: 0.0895 - coeff_determination: 0.9323
Epoch 8/100
7428/7428 [==============================] - 0s 38us/step - loss: 0.0888 - coeff_determination: 0.9347
Epoch 9/100
7428/7428 [==============================] - 0s 35us/step - loss: 0.0878 - coeff_determination: 0.9361
Epoch 10/100
7428/7428 [==============================] - 0s 32us/step - loss: 0.0871 - coeff_determination: 0.9347
Epoch 11/100
7428/7428 [==============================] - 0s 31us/step - loss: 0.0867 - coeff_determination: 0.9357
Epoch 12/100
7428/7428 [==============================] - 0s 43us/step - loss: 0.0859 - coeff_determination: 0.9365
Epoch 13/100
7428/7428 [==============================] - 0s 39us/step - loss: 0.0855 - coeff_determination: 0.9399
Epoch 14/100
7428/7428 [==============================] - 0s 35us/step - loss: 0.0848 - coeff_determination: 0.9380
Epoch 15/100
7428/7428 [==============================] - 0s 34us/step - loss: 0.0845 -

7429/7429 [==============================] - 0s 27us/step - loss: 0.0753 - coeff_determination: 0.9507
Epoch 48/100
7429/7429 [==============================] - 0s 27us/step - loss: 0.0740 - coeff_determination: 0.9508
Epoch 49/100
7429/7429 [==============================] - 0s 26us/step - loss: 0.0744 - coeff_determination: 0.9522
Epoch 50/100
7429/7429 [==============================] - 0s 26us/step - loss: 0.0742 - coeff_determination: 0.9539
Epoch 51/100
7429/7429 [==============================] - 0s 26us/step - loss: 0.0738 - coeff_determination: 0.9504
Epoch 52/100
7429/7429 [==============================] - 0s 27us/step - loss: 0.0737 - coeff_determination: 0.9521
Epoch 53/100
7429/7429 [==============================] - 0s 27us/step - loss: 0.0745 - coeff_determination: 0.9528
Epoch 54/100
7429/7429 [==============================] - 0s 27us/step - loss: 0.0732 - coeff_determination: 0.9504
Epoch 55/100
7429/7429 [==============================] - 0s 27us/step - loss: 0.0725

7429/7429 [==============================] - 0s 27us/step - loss: 0.0864 - coeff_determination: 0.9430
Epoch 18/100
7429/7429 [==============================] - 0s 26us/step - loss: 0.0858 - coeff_determination: 0.9436
Epoch 19/100
7429/7429 [==============================] - 0s 28us/step - loss: 0.0862 - coeff_determination: 0.9442
Epoch 20/100
7429/7429 [==============================] - 0s 27us/step - loss: 0.0855 - coeff_determination: 0.9448
Epoch 21/100
7429/7429 [==============================] - 0s 27us/step - loss: 0.0851 - coeff_determination: 0.9461
Epoch 22/100
7429/7429 [==============================] - 0s 32us/step - loss: 0.0847 - coeff_determination: 0.9462
Epoch 23/100
7429/7429 [==============================] - 0s 34us/step - loss: 0.0838 - coeff_determination: 0.9484
Epoch 24/100
7429/7429 [==============================] - 0s 33us/step - loss: 0.0836 - coeff_determination: 0.9496
Epoch 25/100
7429/7429 [==============================] - 0s 34us/step - loss: 0.0833

7429/7429 [==============================] - 0s 36us/step - loss: 0.0734 - coeff_determination: 0.9554
Epoch 58/100
7429/7429 [==============================] - 0s 31us/step - loss: 0.0755 - coeff_determination: 0.9564
Epoch 59/100
7429/7429 [==============================] - 0s 28us/step - loss: 0.0738 - coeff_determination: 0.9579
Epoch 60/100
7429/7429 [==============================] - 0s 33us/step - loss: 0.0724 - coeff_determination: 0.9564
Epoch 61/100
7429/7429 [==============================] - 0s 35us/step - loss: 0.0726 - coeff_determination: 0.9573
Epoch 62/100
7429/7429 [==============================] - 0s 30us/step - loss: 0.0723 - coeff_determination: 0.9591
Epoch 63/100
7429/7429 [==============================] - 0s 34us/step - loss: 0.0720 - coeff_determination: 0.9595
Epoch 64/100
7429/7429 [==============================] - 0s 33us/step - loss: 0.0719 - coeff_determination: 0.9609
Epoch 65/100
7429/7429 [==============================] - 0s 31us/step - loss: 0.0716

7429/7429 [==============================] - 0s 27us/step - loss: 0.0831 - coeff_determination: 0.9513
Epoch 28/100
7429/7429 [==============================] - 0s 28us/step - loss: 0.0818 - coeff_determination: 0.9525
Epoch 29/100
7429/7429 [==============================] - 0s 28us/step - loss: 0.0817 - coeff_determination: 0.9512
Epoch 30/100
7429/7429 [==============================] - 0s 27us/step - loss: 0.0810 - coeff_determination: 0.9520
Epoch 31/100
7429/7429 [==============================] - 0s 27us/step - loss: 0.0810 - coeff_determination: 0.9522
Epoch 32/100
7429/7429 [==============================] - 0s 27us/step - loss: 0.0811 - coeff_determination: 0.9547
Epoch 33/100
7429/7429 [==============================] - 0s 27us/step - loss: 0.0795 - coeff_determination: 0.9546
Epoch 34/100
7429/7429 [==============================] - 0s 27us/step - loss: 0.0804 - coeff_determination: 0.9556
Epoch 35/100
7429/7429 [==============================] - 0s 28us/step - loss: 0.0799

In [23]:
total_results

[{'fold_0': [{'n_5': [{'ACC': 93.83860726027831,
      'MAE': 234907.58,
      'MAX': array([13500000.]),
      'MEAN': 1624534.1949},
     {'ACC': 93.49708586337428,
      'MAE': 216917.52,
      'MAX': array([13000000.]),
      'MEAN': 1553123.3172},
     {'ACC': 93.40889664446081,
      'MAE': 220259.32,
      'MAX': array([13500000.]),
      'MEAN': 1579596.1228},
     {'ACC': 93.55029710596071,
      'MAE': 247784.26,
      'MAX': array([13500000.]),
      'MEAN': 1732154.0118},
     {'ACC': 93.34933205708805,
      'MAE': 224092.72,
      'MAX': array([13500000.]),
      'MEAN': 1755495.4227}]},
   {'n_10': [{'ACC': 93.46132591390841,
      'MAE': 220862.91,
      'MAX': array([13500000.]),
      'MEAN': 1624534.1949},
     {'ACC': 94.37481550420557,
      'MAE': 209114.96,
      'MAX': array([13000000.]),
      'MEAN': 1553123.3172},
     {'ACC': 93.11108602653219,
      'MAE': 210502.84,
      'MAX': array([13500000.]),
      'MEAN': 1579596.1228},
     {'ACC': 94.0702316770004